# Part 1 - Iterators

## Intro

[Iterators](https://en.wikipedia.org/wiki/Iterator) in Python are a core part of the language and usually hidden / not obvious when used. Iterators are objects that allow for iteration over a collection and statefully keeping track of the iteration progress. Any object in Python can be an iterator by supporting two methods:
- `__next__()` - get the next item in the collection. 
- `__iter__()` - returns an iterator object. Both the **iterable** (a collection supporting iteration) and the iterator object must support this method. When `__iter__()` is called on the iterator, it returns a reference to itself - this allows for reusing the same code with `for` regardless of whether you pass an iterator or an iterable. 

When iteration is complete, the iterator should raise `StopIteration`. 

In [14]:
class ListIterator:
    def __init__(self, data):
        if not isinstance(data, list):
            raise TypeError(f"This iterator is only for lists, not {type(data)}")
        self._data = data
        self._idx = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self._idx >= len(self._data):
            raise StopIteration
        val = self._data[self._idx]
        self._idx += 1
        return val 

In [17]:
li = ListIterator([1,2,3,4,5,6])
for i in li:
    print(i)
 
# Uncoment below to cause a TypeError
# bad_li = ListIterator({1,2,3,4,5})

1
2
3
4
5
6


## `iter()`

Python also provides the `iter()` builtin for creating iterators from iterables. `iter()` works differently depending on use. If only the first argument (`object`) is given, it must be an object supporting either `__iter__()` or `__getitem__()`:


In [22]:
it = iter([1,2,3,4,5,6])
it = iter("1,2,3,4")
it = iter((1,2,3,4))
it = iter({1: 2, 3: 4})

class NotIterable: 
    pass

# Causes TypeError
# it = iter(NotIterable())

If the second argument (`sentinel`) is given, then the first argument is treated as a callable. Each time the iterator's `__next__()` is called, the iterator will call the object with no arguments until a value equal to `sentinel` is produced from the object, at which point the iterator raises `StopIteration`:

In [29]:
class WeirdObj:
    def __init__(self, limit=10):
        self._curr = 0
        self._limit = limit
        
    def __call__(self):
        if self._curr == self._limit:
            return None
        val = self._curr
        self._curr += 1
        return val 
    
for val in iter(WeirdObj(), 6):
    print(val)

0
1
2
3
4
5


## Custom iterators

Iterators don't necessarily need to end, e.g. **infinite iterators**:

In [34]:
class PowersOfTwo:
    def __init__(self):
        self._i = 0
    def __iter__(self):
        return self
    def __next__(self):
        val = 2**self._i
        self._i += 1
        return val
    
it = PowersOfTwo()
print(next(it))
print(next(it))
print(next(it))
print(next(it))
print(next(it))
print(next(it))
print(next(it))
print(next(it))
print(next(it))

1
2
4
8
16
32
64
128
256


You can also implement the `__reversed__()` method on to allow your iterable to be iterated over in reverse order:

In [41]:
class ReversedListIterator:
    def __init__(self, data):
        if not isinstance(data, list):
            raise TypeError(f"This iterator is only for lists, not {type(data)}")
        self._data = data
        self._idx = len(data)-1
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self._idx < 0: 
            raise StopIteration
        val = self._data[self._idx]
        self._idx -= 1
        return val 
    
class MyCustomContainer:
    def __init__(self, data):
        self._data = data
    
    def __iter__(self):
        return ListIterator(self._data)
    
    def __reversed__(self):
        return ReversedListIterator(self._data)
    
con = MyCustomContainer([1,2,3,4,5,6])
for i in iter(con):
    print(i)
    
for i in reversed(con):
    print(i)

1
2
3
4
5
6
6
5
4
3
2
1
